<a href="https://colab.research.google.com/github/Yoonsun222/CapstoneDesign/blob/main/%EC%BA%A1%EC%8A%A4%ED%86%A4%EB%94%94%EC%9E%90%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df1 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/미먼1.csv", encoding = 'euc-kr', parse_dates=['등록일자'])

df2 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/미먼2.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df3 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/미먼3.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df4 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/미먼4.csv", encoding = 'euc-kr', parse_dates=['등록일자'])

df = pd.concat([df1,df2,df3,df4])

df

In [ ]:
df['등록일자'] = df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

df

In [ ]:
weather_df = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/기상청데이터.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
weather_df['등록일자'] = weather_df["등록일자"].dt.strftime("%Y-%m-%d %H:00")
weather_df

In [ ]:

dust_df = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/OBS_부유분진_TIM_20220509183223.csv",encoding = 'euc-kr', parse_dates=['등록일자'] )
dust_df['등록일자'] = dust_df["등록일자"].dt.strftime("%Y-%m-%d %H:00")
dust_df

In [ ]:
data = pd.merge(left = df , right = weather_df, how = "inner", on = "등록일자")
data = pd.merge(left = data , right = dust_df, how = "inner", on = "등록일자")

data.head(100)


data = data.replace(-9999,np.nan)
data = data.replace(-999,np.nan)
data = data.dropna(axis=0)
data = data.drop_duplicates(['시리얼','등록일자'], keep='first')
data = data.sort_values(by=['시리얼','등록일자'] ,ascending=True)


data

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data["시리얼"] = encoder.fit_transform(data["시리얼"])

data = data.reset_index()
data


In [35]:
total_data = data[data.columns[2:]]
total_data = total_data.replace(-9999,np.nan)
total_data = total_data.replace(-999,np.nan)
total_data = total_data.dropna(axis=0)
total_data.head(4)


total_data.columns = ['arduino_temp', 'arduino_hum','arduino_pm25','소음','arduino_VOC','arduino_pm10','datetime','TMP','WSD','REH','SKY','pm25Value']

In [64]:
total_data.to_csv('/content/drive/MyDrive/pre_capstone.csv',index=False)
corr_df= total_data.corr(method='pearson')

corr_df[corr_df.columns[2]]

arduino_temp   -0.263212
arduino_hum     0.146971
arduino_pm25    1.000000
arduino_VOC     0.038431
arduino_pm10    0.999308
TMP             0.071389
WSD            -0.124575
REH             0.061382
SKY            -0.008355
pm25Value       0.143799
ago1_pm25       0.746010
Name: arduino_pm25, dtype: float64

In [117]:
# total_data = pd.read_csv("/content/drive/MyDrive/pre_capstone.csv", encoding = 'euc-kr')

In [73]:
# total_data['arduino_temp'] = (total_data['arduino_temp']/10)-100

In [118]:
condition = []

for idx, i in enumerate(total_data['datetime']):
  date, time = i.split(" ")
  if time != '8:00':
    condition.append(idx)


total_data['datetime'] = total_data.loc[condition, 'datetime']

total_data = total_data.dropna(axis=0)

In [119]:
total_data.head(10)

,arduino_temp,arduino_hum,arduino_pm25,arduino_VOC,arduino_pm10,datetime,TMP,WSD,REH,SKY,pm25Value,ago1_pm25
0,19.9,33.0,78.0,190.0,57.0,2022-04-04 9:00,9.6,0.8,65.0,0.0,53.0,81
1,20.1,32.0,47.0,184.0,34.0,2022-04-04 10:00,11.9,2.0,52.0,0.0,65.0,78
2,20.4,31.0,27.0,168.0,19.0,2022-04-04 11:00,13.8,2.2,38.0,0.0,72.0,47
3,20.3,31.0,64.0,157.0,47.0,2022-04-04 12:00,15.2,3.0,32.0,0.0,57.0,27
4,20.5,30.0,27.0,143.0,19.0,2022-04-04 13:00,16.3,3.5,26.0,0.0,76.0,64
5,20.4,29.0,18.0,130.0,13.0,2022-04-04 14:00,16.2,4.0,30.0,0.0,58.0,27
6,20.3,29.0,5.0,125.0,3.0,2022-04-04 15:00,16.2,3.6,28.0,0.0,64.0,18
7,19.7,27.0,8.0,132.0,5.0,2022-04-04 21:00,9.4,2.0,37.0,0.0,15.0,5
9,19.4,24.0,74.0,131.0,54.0,2022-04-05 9:00,8.8,2.4,53.0,8.0,46.0,40
10,20.1,24.0,81.0,125.0,59.0,2022-04-05 10:00,11.1,1.7,39.0,7.0,61.0,74


In [ ]:
conditionlist = [
  (total_data['ago1_pm25'] <= 30) ,
  (total_data['ago1_pm25'] <= 80) & (total_data['ago1_pm25'] >= 30),
  (total_data['ago1_pm25'] <= 150) & (total_data['ago1_pm25'] >= 80),
  (total_data['ago1_pm25'] >= 150)]
choicelist = [0, 1, 2, 3]
total_data['ago1_pm25_class'] = np.select(conditionlist, choicelist, default='Not Specified')

total_data

In [121]:
X =  total_data[total_data.columns[[0,1,2,4,6,7,8,9,10]]]
Y = total_data[total_data.columns[[-1]]]

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=222)




scaler = MinMaxScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_train = pd.DataFrame(X_scaled_train, columns=X_train.columns, index=X_train.index)

X_scaled_test = scaler.fit_transform(X_test)
X_scaled_test = pd.DataFrame(X_scaled_test, columns=X_test.columns, index=X_test.index)




In [ ]:
# X_scaled_train = np.reshape(X_scaled_train, (2055210, 10 , 2))

In [ ]:
# hist = model.fit(X_train_scaled, Y_train, epochs=10, batch_size=1, validation_split=0.2)

In [135]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier(hidden_layer_sizes=74, activation='relu', solver='adam')
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


0.7253164556962025

In [136]:
import joblib
joblib.dump(model, 'ml_ANN_class_model.pkl')

['ml_ANN_class_model.pkl']

In [ ]:
import joblib

classifer = joblib.load("model.pkl")
new_observation = [[0.945312, 0.528090, 0.009385, 0.2620, 0.676, 0.256410, 0.521127, 1, 0, 0.489051]]

# 샘플의 클래스를 예측합니다.
classifer.predict(new_observation)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"


array([9.46285273])

In [132]:


import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))


훈련   데이터 RMSE: 0.5277130021690084
테스트 데이터 RMSE: 0.5434690881632823


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


In [131]:
from scipy.stats import randint
param_distribs={'hidden_layer_sizes': randint(low=10, high=100), 'solver': ['sgd', 'adam'], 
             'activation': ['tanh', 'relu']}
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(MLPClassifier(), 
                                 param_distributions=param_distribs, n_iter=10, cv=5)
random_search.fit(X_scaled_train, y_train)

print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/li

Best Parameter: {'activation': 'relu', 'hidden_layer_sizes': 74, 'solver': 'adam'}
Best Score: 0.7329
TestSet Score: 0.7122


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
